In [12]:
from collections import Counter
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from stopwords import remove_stopwords

texts = []
textnames = []
folder = '/srv/data/dummyText'
# for file in os.listdir(folder):
path = os.path.join(folder,'A01932.txt')
f = open(path,'r')
data = f.readlines()[0]
texts.append(data)
name = 'A01932.txt'.split('.')[0]
f.close()
 
tokenized = []
#for text in texts:

words = remove_stopwords(texts[0])
print(words[0][1])

# wordcounts = [Counter(t) for t in tokenized]
# df = pd.DataFrame(wordcounts, index=[name in textnames]).fillna(0)

# # normalization turned off
# # sublinear term frequency scaling turned on (takes log of term frequencies and can help to de-emphasize function words like pronouns and articles.
# tfidf = TfidfTransformer(norm=None, sublinear_tf=True)
# results = tfidf.fit_transform(df)

# readable_results = pd.DataFrame(results.toarray(), index=df.index, columns=df.columns)

# # columns are texts
# # use .head(30) to show only the top 30 terms
# # sort using words with highest tfidf scores in A01932 as an example
# readable_results.T.sort_values(by=["A01932"], ascending=False).head(30)


IndexError: list index out of range